In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import read_nml as rnl

from Utils import numerical_utils as nuti
from Utils import constants as Co

print( Co.cpair())

cpair = Co.cpair()
grav= Co.grav()

In [ ]:
# Replace with your path
case='modVortSrc01'
odir = f"../cases/{case}"

nml_path = f"{odir}/atm_in"

nml = rnl.read_namelist(nml_path)
group_name, active = rnl.choose_active_group(nml)

print(f"Active group: {group_name}")
# Access fields robustly
bnd_topo = active.get('bnd_topo')
ncdata = active.get('ncdata')
scale = active.get('scale_dry_air_mass')
use_topo = active.get('use_topo_file')

print("bnd_topo:", bnd_topo)
print("ncdata:", ncdata)
print("scale_dry_air_mass:", scale)
print("use_topo_file:", use_topo)


In [ ]:
print( active )

In [ ]:
#read GW.dat file

%run -i read_gwdat_block.py


In [ ]:
# Open the binary file
with open(f"{odir}/GW_sub.dat", 'rb') as f:
    # Read the dimensions (ncol, nlev) - Fortran writes integers in 4 bytes by default
    ncol, nlev = np.fromfile(f, dtype=np.int32, count=2)
    print(f"Dimensions read from file: ncol={ncol}, nlev={nlev}")
    lat = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    lon = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')
    
    # Read arrays (Here 3D phys ==> two dimensional array). Dims in Fortran order
    zm_2 = np.fromfile(f, dtype=np.float64, count=ncol * nlev).reshape((ncol, nlev), order='F')
    wght_3d = np.fromfile(f, dtype=np.float64, count=ncol * (nlev) ).reshape((ncol, nlev), order='F')
    vort_src = np.fromfile(f, dtype=np.float64, count=ncol  )                                               #.reshape((ncol, nlev+1), order='F')


In [ ]:
print( wght_3d.shape )

i=20_000
plt.plot( wght_3d[i,:], zm_2[i,:] )

In [ ]:

print( ncdata )
print(active['ncdata_type'])


In [ ]:
#
X=xr.open_dataset( ncdata )

In [ ]:
X

In [ ]:
if (active['ncdata_type'] == 'XY_DATA' ):
    dims=np.array( X.dims.values )
    nt = X.sizes['time'] 
    nz = X.sizes['lev'] 
    ny = X.sizes['lat'] 
    nx = X.sizes['lon'] 
    print( X.upwp.values.shape )
    lat_yx = lat.reshape( [nx,ny] )
    sgh_yx = sgh.reshape( [ny,nx] )
    # This reshaping seems to work
    # It is bizarre but we will just go with it.
    zm_yxz = zm.reshape( [ny,nx,nz] )
    pmid_yxz = pmid.reshape( [ny,nx,nz] )
    uu_yxz = uu.reshape( [ny,nx,nz] )
    vv_yxz = vv.reshape( [ny,nx,nz] )
    zeta_yxz = vort_movmtn.reshape( [ny,nx,nz] )
    tau_movmtn_yxz = tau_movmtn.reshape( [ny,nx,nz+1] )
    tau_rdg_yxz = tau.reshape( [ny,nx,nz+1] )
    vort_src_yx = vort_src.reshape( [ny,nx] )
    wght_3d_yxz = wght_3d.reshape( [ny,nx,nz] )
    

In [ ]:
if (active['ncdata_type'] == 'XY_DATA' ):
    # NOte, here the '_m' tag refers to CAM's
    # t,z,y,x dimensions (z optional )
    lon1 = X.lon.values
    lat1 = X.lat.values
    plev = X.lev.values
    upwp_m=X.upwp.values
    vpwp_m=X.vpwp.values
    uu_m=X.U.values
    vv_m=X.V.values
    te_m=X.T.values
    og_m=X.OMEGA.values
    
    mpwp_m=np.sqrt( upwp_m**2 + vpwp_m**2 )
    Topo=xr.open_dataset(bnd_topo)
    htopo_yx=Topo.PHIS.values/9.8
    zeta_m = np.zeros(  (nt,nz,ny,nx) )
    for z in np.arange( nz ):
        zeta_m[0,z,:,:] = nuti.Sphere_Curl2( f_x=uu_m[0,z,:,:], f_y=vv_m[0,z,:,:], lat=lat1, lon=lon1, wrap=True )
        #print( z )

print( htopo_yx.shape )


In [ ]:
movmtn_plaunch = 32500.0
movmtn_psteer = 65000.0
print(plev*100.)
ksteer=np.argmin( np.abs(plev*100.-movmtn_psteer) )
klaunch=np.argmin( np.abs(plev*100.-movmtn_plaunch) )
print(ksteer,klaunch)

In [ ]:
print( np.shape(zm_yxz))
print( np.shape(te_m))

dse_yxz= np.zeros( (ny,nx,nz) )
zma_yxz= np.zeros( (ny,nx,nz) )
for k in np.arange( nz ):
    zma_yxz[:,:,k] = zm_yxz[:,:,k] + htopo_yx[:,:]
    dse_yxz[:,:,k] = grav*(zm_yxz[:,:,k] + htopo_yx[:,:]) + cpair*te_m[0,k,:,:]



In [ ]:
#plt.contour( zma_yxz[:,:,80], levels=51 )
plt.plot( zma_yxz[60,:,0] )
plt.plot( htopo_yx[60,:] )

In [ ]:
fig,axs=plt.subplots( 1,2, figsize=(33,8) )

t,z=0,80
levels=1e-3 * np.linspace( -1,1,num=21)
cmap='bwr'

ax=axs[0]
c=ax.contourf( lon1, lat1, zeta_m[t,z,:,:] , levels= levels, cmap=cmap ) # vort_movmtn[:,90] )
ax.set_title( ' voriticy from Python' )
plt.colorbar(c)
ax=axs[1]
c=ax.contourf( lon1, lat1, zeta_yxz[:,:,z] , levels= levels, cmap=cmap  ) # vort_movmtn[:,90] )
ax.set_title( 'vorticity from Fortran' )
plt.colorbar(c)


In [ ]:
vosrc_yx=np.zeros( ( ny, nx ))

In [ ]:
for i in np.arange( nx ):
    for j in np.arange( ny ):
        vosrc_yx[ j,i]= np.sum( wght_3d_yxz[j,i,:] * np.abs(zeta_yxz[j,i,:] ) ) / np.sum( wght_3d_yxz[j,i,:] )

In [ ]:
fig,axs=plt.subplots(1,2,figsize=(20,7))
ax=axs[0]
c=ax.contourf( vosrc_yx,levels=21 )
plt.colorbar(c)
ax=axs[1]
c=ax.contourf( vort_src_yx,levels=21 )
plt.colorbar(c)



In [ ]:
plev=X.lev.values
zlev= -7 *np.log( plev/1_000.)

In [ ]:
tau_plot=tau_movmtn # +tau

mflev=0.1*np.linspace(0,5,num=26)
ztlev=1e-3 * np.linspace( -1,1,num=21)
mflev=2*np.linspace(0,0.1,num=41)

z=11
z=25
#z=40
#z=85
zsrc=60
print(zlev[z])

lonrn=[0,150]
latrn=[-90,-40]

lonrn=[0,360]
latrn=[-80,-0]


#mflev=0.01*np.linspace(0.,2.5,num=51)
fig,axs=plt.subplots(1,3,figsize=(30,5))
ax=axs[0]
#co=ax.tricontourf( lon,lat, tau_plot.T[z,:], levels=mflev )
co=ax.contourf( lon1,lat1, 9.8*tau_movmtn_yxz[:,:,z], levels=mflev , cmap='bwr' )
to=ax.contour( lon1,lat1, htopo_yx, levels=[1,100000],colors='white')
ax.set_xlim( lonrn )
ax.set_ylim( latrn )
plt.colorbar(co)

ax=axs[1]
co=ax.contourf( lon1,lat1, mpwp_m[0,z,:,:], levels=mflev , cmap='bwr'  )
to=ax.contour( lon1,lat1, htopo_yx, levels=[1,100000],colors='white')
ax.set_xlim( lonrn )
ax.set_ylim( latrn )
plt.colorbar(co)

ax=axs[2]
#co=ax.contourf( lon1,lat1, np.average( np.abs(zeta_yxz[:,:,nz-30:nz-10]), axis=2 ), levels=ztlev, cmap='bwr'  )
#co=ax.contourf( lon1,lat1, np.average( zeta_yxz[:,:,65:72], axis=2 ), levels=ztlev/2, cmap='bwr'  )
co=ax.contourf( lon1,lat1, 9.8*vort_src_yx, levels=mflev, cmap='bwr'  )


#co=ax.contourf( lon1,lat1, np.average( np.abs(zeta_yxz[:,:,65:75]), axis=2 ), levels=ztlev/2, cmap='bwr'  )
#co=ax.contourf( lon1,lat1, np.abs(zeta_yxz[:,:,70]), levels=ztlev/2, cmap='bwr'  )
to=ax.contour( lon1,lat1, htopo_yx, levels=[1,100000],colors='white')
ax.set_xlim( lonrn )
ax.set_ylim( latrn )
plt.colorbar(co)


print( plev[65:72] )
print( lat1[40])

In [ ]:
tau_plot=tau_movmtn # +tau

mflev=0.1*np.linspace(0,5,num=26)
ztlev=1e-3 * np.linspace( -1,1,num=21)
mflev=2*np.linspace(0,0.1,num=41)

xin1,yin1=np.arange(nx),np.arange(ny)
xirn=[0,75]
yirn=[0,50]


z=11
#z=40
#z=85
zsrc=60
print(zlev[z])

#mflev=0.01*np.linspace(0.,2.5,num=51)
fig,axs=plt.subplots(1,3,figsize=(30,5))
ax=axs[0]
#co=ax.tricontourf( lon,lat, tau_plot.T[z,:], levels=mflev )
co=ax.contourf( xin1,yin1, 9.8*tau_movmtn_yxz[:,:,z], levels=mflev , cmap='bwr' )
to=ax.contour( xin1,yin1, htopo_yx, levels=[1,100000],colors='white')
ax.set_xlim( xirn )
ax.set_ylim( yirn )
plt.colorbar(co)

ax=axs[1]
co=ax.contourf( xin1,yin1, mpwp_m[0,z,:,:], levels=mflev , cmap='bwr'  )
to=ax.contour( xin1,yin1, htopo_yx, levels=[1,100000],colors='white')
ax.set_xlim( xirn )
ax.set_ylim( yirn )
plt.colorbar(co)

ax=axs[2]
#co=ax.contourf( xin1,yin1, np.average( np.abs(zeta_yxz[:,:,nz-30:nz-10]), axis=2 ), levels=ztlev, cmap='bwr'  )
#co=ax.contourf( xin1,yin1, np.average( zeta_yxz[:,:,65:72], axis=2 ), levels=ztlev/2, cmap='bwr'  )
co=ax.contourf( xin1,yin1, 9.8*vort_src_yx, levels=mflev, cmap='bwr'  )


#co=ax.contourf( xin1,yin1, np.average( np.abs(zeta_yxz[:,:,65:75]), axis=2 ), levels=ztlev/2, cmap='bwr'  )
#co=ax.contourf( xin1,yin1, np.abs(zeta_yxz[:,:,70]), levels=ztlev/2, cmap='bwr'  )
to=ax.contour( xin1,yin1, htopo_yx, levels=[1,100000],colors='white')
ax.set_xlim( xirn )
ax.set_ylim( yirn )
plt.colorbar(co)


print( plev[65:72] )
print( lat1[40])

In [ ]:
z=85
plt.scatter( vort_src_yx.flatten() , tau_movmtn_yxz[:,:,z].flatten() )

In [ ]:
tau_plot=tau  #_movmtn # +tau

mflev=0.1*np.linspace(0,5,num=26)
ztlev=1e-3 * np.linspace( -1,1,num=21)
mflev=8*np.linspace(0,0.1,num=41)

z=11
z=40
zsrc=60
print(zlev[z])

#mflev=0.01*np.linspace(0.,2.5,num=51)
fig,axs=plt.subplots(1,3,figsize=(30,5))
ax=axs[0]
#co=ax.tricontourf( lon,lat, tau_plot.T[z,:], levels=mflev )
co=ax.contourf( lon1,lat1, 9.8*tau_rdg_yxz[:,:,z], levels=mflev , cmap='gist_ncar' , extend='both' )
to=ax.contour( lon1,lat1, htopo_yx, levels=[1,100000],colors='white')
plt.colorbar(co)
ax=axs[1]
co=ax.contourf( lon1,lat1, mpwp_m[0,z,:,:], levels=mflev , cmap='gist_ncar', extend='both'   )
to=ax.contour( lon1,lat1, htopo_yx, levels=[1,100000],colors='white')
plt.colorbar(co)

ax=axs[2]
#co=ax.contourf( lon1,lat1, np.average( np.abs(zeta_yxz[:,:,nz-30:nz-10]), axis=2 ), levels=ztlev, cmap='bwr'  )
co=ax.contourf( lon1,lat1, np.average( zeta_yxz[:,:,65:74], axis=2 ), levels=ztlev/2, cmap='bwr'  )
#co=ax.contourf( lon1,lat1, np.average( np.abs(zeta_yxz[:,:,65:75]), axis=2 ), levels=ztlev/2, cmap='bwr'  )
#co=ax.contourf( lon1,lat1, np.abs(zeta_yxz[:,:,70]), levels=ztlev/2, cmap='bwr'  )
to=ax.contour( lon1,lat1, htopo_yx, levels=[1,100000],colors='white')
plt.colorbar(co)


print( plev[65:72] )
print( lat1[40])

In [ ]:
plt.scatter(9.8*tau_movmtn_yxz[:,:,z],mpwp_m[0,z,:,:] )

In [ ]:
print( np.shape(zeta_yxz))

lat_plot=-80.
print( np.argmin( np.abs(lat1 - lat_plot) ) )

In [ ]:


pv_yxz = np.zeros( (ny,nx,nz) )
dsedz_yxz = np.zeros( (ny,nx,nz) )
duvdz_yxz = np.zeros( (ny,nx,nz) )

for k in np.arange( start=1, stop=nz-1 ):
    dsedz_yxz[:,:,k] = (1./cpair)*(dse_yxz[:,:,k-1] - dse_yxz[:,:,k+1]) / (zma_yxz[:,:,k-1] - zma_yxz[:,:,k+1])
    duvdz_yxz[:,:,k] = np.sqrt( ( (uu_yxz[:,:,k-1] - uu_yxz[:,:,k+1]) / (zma_yxz[:,:,k-1] - zma_yxz[:,:,k+1]) )**2 
                               + 0.*( (vv_yxz[:,:,k-1] - vv_yxz[:,:,k+1]) / (zma_yxz[:,:,k-1] - zma_yxz[:,:,k+1]) )**2 )
                        

pv_yxz = zeta_yxz * dsedz_yxz


In [ ]:

#mflev=0.1*np.linspace(0,5,num=26)
mflev=0.01*np.linspace(0.,2.5,num=51)

shrlev=3e-2 * np.linspace( 0,1,num=21)
ztlev=1e-3 * np.linspace( -1,1,num=21)
pvlev=5e-6 * np.linspace( -1,1,num=21)


lon_plot,lat_plot=40.,-75.
#lon_plot,lat_plot=320.,52.
#lon_plot,lat_plot=175.,-60.

y = np.argmin( np.abs(lat1 - lat_plot) ) 
x = np.argmin( np.abs(lon1 - lon_plot) ) 
print( lon1[y] )

Z2d = zma_yxz[y, :, :].T   # shape (nz, nx)  # levels × longitude
Lon = np.broadcast_to(lon1, Z2d.shape) 


print( Lon.T.shape )
print( Z2d.shape )
fig,axs=plt.subplots(1,3,figsize=(24,6))
axs=axs.flatten()

ax=axs[0]
c=ax.contourf(Lon, Z2d , tau_movmtn_yxz[y,:,1:].T, levels=mflev/2,cmap='bwr' )
#l=ax.contour(Lon, Z2d , 0.1*mpwp_m[0,:,y,:], levels=[0.005,0.01,0.05,.1,.2,.5,1.], colors='black' )

ax.set_ylim(0,25000)
plt.colorbar(c,ax=ax)
ax.set_title( f"param. mom flux" )



ax=axs[1]
#c=ax.contourf( Lon, Z2d ,zeta_m[0,:,y,:], levels=0.1*ztlev, cmap='bwr' )
#c=ax.contourf( Lon, Z2d ,pv_yxz[y,:,:].T, levels=pvlev/10, cmap='bwr' )
c=ax.contourf(Lon, Z2d ,  0.1*mpwp_m[0,:,y,:] , levels=mflev,cmap='bwr' )
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
ax.set_ylim(0,25000)
plt.colorbar(c,ax=ax)
ax.set_title( f"resolved mom flux" )




ax=axs[2]
#c=ax.contourf( Lon, Z2d ,uu_m[0,:,y,:], levels=np.linspace(-120,120,num=31), cmap='bwr' )
#c=ax.contourf( Lon, Z2d ,og_m[0,:,y,:], levels=np.linspace(-1,1,num=31), cmap='bwr' )
c=ax.contourf( Lon, Z2d ,zeta_yxz[y,:,:].T, levels=0.5*ztlev, cmap='bwr' )
ax.set_ylim(0,25000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"vorticity" )



"""
ax=axs[3]
#c=ax.contourf( Lon, Z2d , dse_yxz[y,:,:].T / cpair , levels=np.linspace(270,390,num=31)-40., cmap='bwr' )
c=ax.contourf( Lon, Z2d ,uu_m[0,:,y,:], levels=np.linspace(-120,120,num=31), cmap='bwr' )

ax.set_ylim(0,45000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"U" )

ax=axs[4]
#c=ax.contourf( Lon, Z2d , dse_yxz[y,:,:].T / cpair , levels=np.linspace(270,390,num=31)-40., cmap='bwr' )
c=ax.contourf( Lon, Z2d ,duvdz_yxz[y,:,:].T, levels=shrlev, cmap='gist_ncar' )

ax.set_ylim(0,5000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"dU/dz" )

ax=axs[5]
c=ax.contourf( Lon, Z2d , dse_yxz[y,:,:].T / cpair , levels=np.linspace(270,390,num=31)-40., cmap='bwr' )


ax.set_ylim(0,15000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"dse" )

ax=axs[6]
voshrlev=1.e-6*np.linspace( -4,4,num=41) 
c=ax.contourf( Lon, Z2d , (zeta_yxz*duvdz_yxz)[y,:,:].T  , levels=voshrlev, cmap='bwr' )


ax.set_ylim(0,15000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
tt=r"$\zeta~\partial_z |\mathbf{U}| $"

ax.set_title( f"{tt}" )

ax=axs[7]
c=ax.contourf( Lon, Z2d ,pv_yxz[y,:,:].T, levels=pvlev/10, cmap='bwr' )
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
ax.set_ylim(0,15000)
plt.colorbar(c,ax=ax)
ax.set_title( f"Pot. vorticity" )

"""



In [ ]:

#mflev=0.1*np.linspace(0,5,num=26)
mflev=0.01*np.linspace(0.,2.5,num=51)

shrlev=3e-2 * np.linspace( 0,1,num=21)
ztlev=1e-3 * np.linspace( -1,1,num=21)
pvlev=5e-6 * np.linspace( -1,1,num=21)


lon_plot,lat_plot=40.,-75.
#lon_plot,lat_plot=320.,52.
#lon_plot,lat_plot=175.,-60.

y = np.argmin( np.abs(lat1 - lat_plot) ) 
x = np.argmin( np.abs(lon1 - lon_plot) ) 
print( lon1[y] )

Z2d = zma_yxz[y, :, :].T   # shape (nz, nx)  # levels × longitude
Lon = np.broadcast_to(lon1, Z2d.shape) 


print( Lon.T.shape )
print( Z2d.shape )
fig,axs=plt.subplots(2,4,figsize=(48,11))
axs=axs.flatten()

ax=axs[0]
c=ax.contourf(Lon, Z2d , tau_movmtn_yxz[y,:,1:].T, levels=mflev )
l=ax.contour(Lon, Z2d , 0.1*mpwp_m[0,:,y,:], levels=[0.005,0.01,0.05,.1,.2,.5,1.], colors='white' )

ax.set_ylim(0,45000)
plt.colorbar(c,ax=ax)



ax=axs[1]
#c=ax.contourf( Lon, Z2d ,zeta_m[0,:,y,:], levels=0.1*ztlev, cmap='bwr' )
c=ax.contourf( Lon, Z2d ,zeta_yxz[y,:,:].T, levels=0.5*ztlev, cmap='bwr' )
#c=ax.contourf( Lon, Z2d ,pv_yxz[y,:,:].T, levels=pvlev/10, cmap='bwr' )
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
ax.set_ylim(0,15000)
plt.colorbar(c,ax=ax)
ax.set_title( f"vorticity" )




ax=axs[2]
#c=ax.contourf( Lon, Z2d ,uu_m[0,:,y,:], levels=np.linspace(-120,120,num=31), cmap='bwr' )
c=ax.contourf( Lon, Z2d ,og_m[0,:,y,:], levels=np.linspace(-1,1,num=31), cmap='bwr' )
ax.set_ylim(0,15000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"omega" )

ax=axs[3]
#c=ax.contourf( Lon, Z2d , dse_yxz[y,:,:].T / cpair , levels=np.linspace(270,390,num=31)-40., cmap='bwr' )
c=ax.contourf( Lon, Z2d ,uu_m[0,:,y,:], levels=np.linspace(-120,120,num=31), cmap='bwr' )

ax.set_ylim(0,45000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"U" )

ax=axs[4]
#c=ax.contourf( Lon, Z2d , dse_yxz[y,:,:].T / cpair , levels=np.linspace(270,390,num=31)-40., cmap='bwr' )
c=ax.contourf( Lon, Z2d ,duvdz_yxz[y,:,:].T, levels=shrlev, cmap='gist_ncar' )

ax.set_ylim(0,5000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"dU/dz" )

ax=axs[5]
c=ax.contourf( Lon, Z2d , dse_yxz[y,:,:].T / cpair , levels=np.linspace(270,390,num=31)-40., cmap='bwr' )


ax.set_ylim(0,15000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
ax.set_title( f"dse" )

ax=axs[6]
voshrlev=1.e-6*np.linspace( -4,4,num=41) 
c=ax.contourf( Lon, Z2d , (zeta_yxz*duvdz_yxz)[y,:,:].T  , levels=voshrlev, cmap='bwr' )


ax.set_ylim(0,15000)
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
plt.colorbar(c,ax=ax)
tt=r"$\zeta~\partial_z |\mathbf{U}| $"

ax.set_title( f"{tt}" )

ax=axs[7]
c=ax.contourf( Lon, Z2d ,pv_yxz[y,:,:].T, levels=pvlev/10, cmap='bwr' )
ax.plot( lon1, Z2d[83,:] , color='black' )
ax.plot( lon1, Z2d[ksteer,:] , color='black' )
ax.plot( lon1, Z2d[klaunch,:] , color='black' )
ax.set_ylim(0,15000)
plt.colorbar(c,ax=ax)
ax.set_title( f"Pot. vorticity" )





In [ ]:
#plt.plot( mpwp_m[0,:,y,x] , Z2d[:,x])

#plt.plot( (pmid_yxz*zeta_yxz)[y,x,:] , pmid_yxz[y,x,:])  #Z2d[:,x])
#plt.plot( (pmid_yxz*zeta_yxz*duvdz_yxz)[y,x,:] , pmid_yxz[y,x,:])  #Z2d[:,x])
#plt.plot( (pmid_yxz*zeta_yxz*duvdz_yxz)[y,x,:] , zm_yxz[y,x,:])  #Z2d[:,x])
plt.plot( (pmid_yxz*zeta_yxz)[y,x,:] , zm_yxz[y,x,:])  #Z2d[:,x])
#plt.plot( (pmid_yxz*pv_yxz)[y,x,:] , pmid_yxz[y,x,:])  #Z2d[:,x])
#plt.ylim(100_000.,0.)



In [ ]:
#plt.plot( mpwp_m[0,:,y,x] , Z2d[:,x])

plt.plot( (pmid_yxz*zeta_yxz)[y,x,:] , pmid_yxz[y,x,:])  #Z2d[:,x])
#plt.plot( (pmid_yxz*zeta_yxz*duvdz_yxz)[y,x,:] , pmid_yxz[y,x,:])  #Z2d[:,x])
#plt.plot( (pmid_yxz*zeta_yxz*duvdz_yxz)[y,x,:] , zm_yxz[y,x,:])  #Z2d[:,x])
#plt.plot( (pmid_yxz*pv_yxz)[y,x,:] , pmid_yxz[y,x,:])  #Z2d[:,x])
plt.ylim(100_000.,0.)



In [ ]:
int1 = np.sum( (pmid_yxz*np.abs(zeta_yxz))[y,x,:] )/np.sum( np.abs(zeta_yxz)[y,x,:] )

In [ ]:
print(int1)